In [1]:
import findspark
from pyspark.sql.session import SparkSession
from pyspark import SparkContext, SparkConf

# Inicializando Spark
findspark.init("/usr/spark-3.5.1/")

spark = (
    SparkSession.builder.appName("sparksubmit_test_app")
    .config("spark.sql.warehouse.dir", "hdfs:///user/hive/warehouse")
    .config("spark.sql.catalogImplementation", "hive")
    .getOrCreate()
)

# criar um contexto de sessão do spark (cria um "programa")
sc = SparkContext.getOrCreate()



def main():

    df = spark.read.text("hdfs://spark-master:9000/datasets/")  # noqa: F841

    # variável recebe o caminho que aponta para uma arquivo de texto
    file_path = "/datasets/*.txt"
    # leitura do arquivo de texto pelo programa spark
    words = sc.textFile(f"{file_path}").flatMap(lambda line: line.split(" "))
    # contagem de palavras utilizando a sintaxe facilitada do pyspark
    wordCounts = words.map(lambda word: (word, 1)).reduceByKey(lambda a,b:a +b)
    # salvando arquivo com resultado da execução
    wordCounts.saveAsTextFile(f"{'/'.join(file_path.split('/')[:-1])}/word_count")
   
    print(wordCounts.count())
    print(wordCounts.countApproxDistinct())


if __name__ == "__main__":
    main()



spark.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/07 23:27:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/07 23:27:25 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


Py4JJavaError: An error occurred while calling o45.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input Pattern hdfs://spark-master:9000/datasets/*.txt matches 0 files
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.api.java.JavaRDDLike.partitions(JavaRDDLike.scala:61